In [2]:
import pickle
import qg
import spacy
from chunking import ChunkPipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from qg import QGPipeline
from sklearn.feature_extraction.text import CountVectorizer

Get transcript chunks and  get slide chunks

In [3]:
# get the whisper chunks
with open("experiments/qg/comp3074_lecture_2.pkl", "rb") as file:
    whisper_chunks = pickle.load(file)['chunks']

# get the slide chunks with timestamps
with open('slide_chunks.pkl', 'rb') as file:
    slide_chunks = pickle.load(file)

# generate trasncript chunks
qg_model = qg.Model.DISCORD
chunk_pipe = ChunkPipeline(qg_model)
transcript_chunks = chunk_pipe(whisper_chunks,2301)

# get last endtime in the slide chunkss
endtime = slide_chunks[-1][2]

Function to compute the relevant chunk scores, we can pass different similarity functions to this method in order to experiment with the different similarity and embedding methods

In [4]:

def compute_similarity_scores(similarity_function):
    i = 0
    relevant_chunks = []

    for j, chunk in enumerate(transcript_chunks):
        if chunk['timestamp'][0] < endtime:
            list_of_slide_indices = []

            while i < len(slide_chunks):
                list_of_slide_indices.append(i)

                if chunk['timestamp'][1] <= slide_chunks[i][2]:
                    transcript_chunk_text = chunk['text']
                    print(f"The index of this transcript chunk is: {j}")

                    print(f"the slide indices that need to be combined are: {list_of_slide_indices}")
                    
                    slide_text = ""
                    for index in list_of_slide_indices:
                        slide_text += slide_chunks[index][0]
                    print(slide_text)

                    cosine_sim = similarity_function(transcript_chunk_text, slide_text)
                    relevant_chunks.append(cosine_sim)
                    break
                i += 1

    return relevant_chunks

Using Cosine similarity and tfidf vectoriser

In [5]:
def compute_cosine_similarity(text1, text2):
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform([text1, text2])
    cosine_sim = cosine_similarity(vectors)
    return cosine_sim[0, 1]

In [6]:
print(compute_similarity_scores(compute_cosine_similarity))

The index of this transcript chunk is: 0
the slide indices that need to be combined are: [0, 1, 2]
COMP3074
Human-AI Interaction
Lecture 2:
Natural Language 
Processing
Dr Jeremie Clos
Dr Joel Fischer
Lecture 2 – Part 1
An Overview of Natural Language Processing
▪ Language is at the key of interaction
▪ Interaction between people (spoken/sign languages, writing systems)
▪ Interaction with machines (command line interface, voice interface, etc.)
▪ Interaction between machines (networking protocols)
▪ Interaction with animals?
▪ Etc.
▪ Advances in speech recognition and speech generation (thanks to 
deep learning1) brought natural language interfaces back!
▪ Natural language → languages which naturally evolved over time
▪ English, French, Chinese, etc. → natural languages
▪ Python, C++ → not natural languages, designed for a purpose
Language and interaction
1: Learn more at https://deepmind.com/blog/article/wavenet-generative-model-raw-audio

The index of this transcript chunk is: 1
the 

Using jaccard Similarity and count vectoriser

In [7]:
def compute_jaccard_similarity(text1, text2):
    # Create a CountVectorizer to convert text to a bag-of-words representation
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform([text1, text2]).toarray()

    # Compute Jaccard similarity between the two text vectors
    intersection = sum(min(v1, v2) for v1, v2 in zip(vectors[0], vectors[1]))
    union = sum(max(v1, v2) for v1, v2 in zip(vectors[0], vectors[1]))

    jaccard_sim = intersection / union if union != 0 else 0
    return jaccard_sim

In [8]:
print(compute_similarity_scores(compute_jaccard_similarity))

The index of this transcript chunk is: 0
the slide indices that need to be combined are: [0, 1, 2]
COMP3074
Human-AI Interaction
Lecture 2:
Natural Language 
Processing
Dr Jeremie Clos
Dr Joel Fischer
Lecture 2 – Part 1
An Overview of Natural Language Processing
▪ Language is at the key of interaction
▪ Interaction between people (spoken/sign languages, writing systems)
▪ Interaction with machines (command line interface, voice interface, etc.)
▪ Interaction between machines (networking protocols)
▪ Interaction with animals?
▪ Etc.
▪ Advances in speech recognition and speech generation (thanks to 
deep learning1) brought natural language interfaces back!
▪ Natural language → languages which naturally evolved over time
▪ English, French, Chinese, etc. → natural languages
▪ Python, C++ → not natural languages, designed for a purpose
Language and interaction
1: Learn more at https://deepmind.com/blog/article/wavenet-generative-model-raw-audio

The index of this transcript chunk is: 1
the 

Using word embeddings instead of tf_idf

In [9]:
!python -m spacy download en_core_web_md

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 5.4 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [10]:
def compute_cosine_similarity_word_embeddings(text1, text2):
    # Load spaCy model with word embeddings
    nlp = spacy.load("en_core_web_md")

    # Get the word embeddings for each text
    embeddings1 = nlp(text1).vector.reshape(1, -1)
    embeddings2 = nlp(text2).vector.reshape(1, -1)

    # Compute cosine similarity between the two vectors
    cosine_sim = cosine_similarity(embeddings1, embeddings2)[0, 0]
    return cosine_sim

In [11]:
print(compute_similarity_scores(compute_cosine_similarity_word_embeddings))

The index of this transcript chunk is: 0
the slide indices that need to be combined are: [0, 1, 2]
COMP3074
Human-AI Interaction
Lecture 2:
Natural Language 
Processing
Dr Jeremie Clos
Dr Joel Fischer
Lecture 2 – Part 1
An Overview of Natural Language Processing
▪ Language is at the key of interaction
▪ Interaction between people (spoken/sign languages, writing systems)
▪ Interaction with machines (command line interface, voice interface, etc.)
▪ Interaction between machines (networking protocols)
▪ Interaction with animals?
▪ Etc.
▪ Advances in speech recognition and speech generation (thanks to 
deep learning1) brought natural language interfaces back!
▪ Natural language → languages which naturally evolved over time
▪ English, French, Chinese, etc. → natural languages
▪ Python, C++ → not natural languages, designed for a purpose
Language and interaction
1: Learn more at https://deepmind.com/blog/article/wavenet-generative-model-raw-audio

The index of this transcript chunk is: 1
the 

In [13]:
qg_pipe = QGPipeline(qg_model)
text_chunks = [chunk["text"] for chunk in transcript_chunks]
generated_questions = qg_pipe(text_chunks)
print(generated_questions)

/Users/zawaralilearning/Documents/University/Year 4/DISS/mmqg/.venv/lib/python3.10/site-packages/transformers/generation/utils.py:1355: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['What is the concept of an NLP pipeline?', 'What was the number of students in the room?', 'What does the field of natural language processing do?', 'What does natural language generation do?', 'What is the evolution of NLP?', 'What was the era of the classical machine learning algorithms?', 'What is the current state of NLP research?', 'What does the system do when it detects a question?', 'What is the use of NLP?', 'What does a document look like?', 'What does tokenization do?', 'What does part of speech mean?', 'What is the goal of standardization?', 'What does the last step do?', 'What was the name of the paper?', 'What was the basis for the word spelling correction in Microsoft Word?', 'What does the process of the process use?', 'What will the lab be doing?']
